In [2]:
from transformers import Wav2Vec2FeatureExtractor, HubertModel
import torchaudio
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import os
import pickle
import requests

In [3]:
# 1. HuBERT 특성 추출기 정의
class HuBERTFeatureExtractor:
    def __init__(self, model_name="facebook/hubert-base-ls960"):
        # HuBERT 모델과 프로세서 초기화
        self.processor = Wav2Vec2FeatureExtractor.from_pretrained(model_name)
        self.model = HubertModel.from_pretrained(model_name)
        self.model.eval()  # 평가 모드로 설정

    def load_audio(self, audio_file):
        # 오디오 파일 로드
        waveform, sample_rate = torchaudio.load(audio_file, format="wav")
        
        return waveform, sample_rate

    def preprocess_audio(self, waveform, sample_rate, target_sample_rate=16000, max_length=10):
        # 모노로 변환
        if waveform.size(0) > 1:
            waveform = waveform.mean(dim=0, keepdim=True)
        
        # 샘플링 레이트 변환
        if sample_rate != target_sample_rate:
            resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=target_sample_rate)
            waveform = resampler(waveform)
            
        max_samples = target_sample_rate * max_length
        if waveform.size(1) > max_samples:
            waveform = waveform[:, :max_samples]
            
        return waveform

    def extract_features(self, audio_file):
        # 오디오 로드 및 전처리
        waveform, sample_rate = self.load_audio(audio_file)
        waveform = self.preprocess_audio(waveform, sample_rate)
        
        # 입력 차원 확인 및 조정
        if waveform.dim() == 1:
            waveform = waveform.unsqueeze(0)
        elif waveform.dim() == 2:
            if waveform.size(0) > 1:
                waveform = waveform.mean(dim=0, keepdim=True)
        elif waveform.dim() == 3:
            waveform = waveform.squeeze(0)
            if waveform.size(0) > 1:
                waveform = waveform.mean(dim=0, keepdim=True)
        
        # 특성 추출
        inputs = self.processor(waveform, sampling_rate=16000, return_tensors="pt", padding=True)
        input_values = inputs.input_values
        
        # 불필요한 차원 제거
        input_values = input_values.squeeze(1)  # (batch_size=1, sequence_length)
        
        with torch.no_grad():
            outputs = self.model(input_values)
            
        features = outputs.last_hidden_state
        
        return features

In [4]:
# 2. 데이터셋 클래스 정의
class EmotionDataset:
    def __init__(self, file_paths, feature_extractor):
        self.file_paths = file_paths
        self.feature_extractor = feature_extractor

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        audio_path = self.file_paths[idx]
        features = self.feature_extractor.extract_features(audio_path)
        features = features.squeeze(0)  # (sequence_length, hidden_size)
        return features

In [5]:
# 3. Transformer 모델 정의
class EmotionTransformer(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(EmotionTransformer, self).__init__()
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=input_dim, nhead=16),
            num_layers=8
        )
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x, src_key_padding_mask=None):
        # Transformer를 통해 특성 추출
        x = self.transformer(x, src_key_padding_mask=src_key_padding_mask)
        x = x[-1]  # Sequence의 평균을 사용
        output = self.fc(x)
        return output

In [6]:
def predict_emotion(dataloader, model, label_encoder):
    device = torch.device("mps")
    model = model.to(device)

    predictions = []
    with torch.no_grad():
        for features in dataloader:
            features = features.to(device)
            outputs = model(features)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())

    # 예측값을 레이블로 변환
    decoded_predictions = label_encoder.inverse_transform(predictions)
    return decoded_predictions

In [7]:
def collate_fn(batch):
    features = [item for item in batch]
    features = torch.nn.utils.rnn.pad_sequence(features, batch_first=True)  # [batch_size, sequence_length, feature_dim]
    return features

In [8]:
# TMDB API 키
TMDB_API_KEY = "df9a0caaf2a07ee6babd7024a6accaf8"
    
EMOTION_TO_GENRE = {
    '기쁨': 35,  # Comedy
    '슬픔': 18,  # Drama
    '분노': 53,  # Thriller
    '불안': 27,  # Horror
    '상처': 80,  # Crime
    '당황': 28,  # Action
    '중립': 10751,  # Family
}

def get_recommendations(emotion, result_num=10, api_key=TMDB_API_KEY):
    # 감정 매핑 확인
    genre_id = EMOTION_TO_GENRE.get(emotion)
    if not genre_id:
        return f"'{emotion}'에 해당하는 추천 장르가 없습니다. 감정을 다시 입력해주세요."

    # TMDB Discover API 호출
    url = f"https://api.themoviedb.org/3/discover/movie"
    params = {
        "api_key": api_key,
        "with_genres": genre_id,
        "sort_by": "popularity.desc",  # 인기 순으로 정렬
        "language": "ko-KR",          # 한국어 결과
        "vote_average.gte": 7.0,      # 평점 7 이상
    }

    response = requests.get(url, params=params)
    if response.status_code != 200:
        return f"TMDB API 호출 실패: {response.status_code}"

    data = response.json()
    results = data.get("results", [])

    if not results:
        return f"'{emotion}'에 맞는 추천 콘텐츠를 찾을 수 없습니다."

    # 추천 콘텐츠 추출
    recommendations = []
    for movie in results[:result_num]:  # 상위 N개만 추출
        recommendations.append({
            "title": movie.get("title"),
            "overview": movie.get("overview"),
            "vote_average": movie.get("vote_average"),
            "release_date": movie.get("release_date"),
        })

    return recommendations

In [38]:
model = torch.load('./model.pth')
model.eval()

/var/folders/4h/8jkvbs3n7tlcqj21nw5kbbqm0000gn/T/ipykernel_45105/2282715700.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('./model.pth')


EmotionTransformer(
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-7): 8 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (linear1): Linear(in_features=768, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=768, bias=True)
        (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (fc): Linear(in_features=768, out_features=7, bias=True)
)

In [41]:
test_audio_file = ["./dataset/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/원천데이터/1.감정/4.불안/0014_G2A3E4S0C0_ASH/0014_G2A3E4S0C0_ASH_000002.wav"]  # 테스트할 오디오 파일 경로

with open("./label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

# Feature extractor 설정
feature_extractor = HuBERTFeatureExtractor()

# 테스트 데이터셋 및 DataLoader 생성
test_dataset = EmotionDataset(test_audio_file, feature_extractor)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)

# 감정 예측
predicted_emotions = predict_emotion(test_dataloader, model, label_encoder)

# 결과 출력
for audio_file, emotion in zip(test_audio_file, predicted_emotions):
    print(f"Audio File: {os.path.basename(audio_file)} -> Predicted Emotion: {emotion}")

Audio File: 0014_G2A3E4S0C0_ASH_000002.wav -> Predicted Emotion: 불안


In [42]:
recommendations = get_recommendations(emotion, 3)

if isinstance(recommendations, str):
    print(recommendations)  # 에러 메시지 출력
else:
    print(f"'{emotion}'에 맞는 추천 콘텐츠:")
    for idx, movie in enumerate(recommendations, 1):
        print(f"\n{idx}. 제목: {movie['title']}")
        print(f"   개봉일: {movie['release_date']}")
        print(f"   평점: {movie['vote_average']}")
        print(f"   줄거리: {movie['overview']}")

'불안'에 맞는 추천 콘텐츠:

1. 제목: 서브스턴스
   개봉일: 2024-09-07
   평점: 7.149
   줄거리: 더 나은 버전의 당신을 꿈꿔본 적 있나요? 당신의 인생을 바꿔줄 신제품 ‘서브스턴스’. ‘서브스턴스’는 또 다른 당신을 만들어냅니다. 새롭고, 젊고, 더 아름답고, 더 완벽한 당신을. 단 한가지 규칙, 당신의 시간을 공유하면 됩니다. 당신을 위한 일주일, 새로운 당신을 위한 일주일, 각각 7일간의 완벽한 밸런스. 쉽죠? 균형을 존중한다면… 무엇이 잘못될 수 있을까요?

2. 제목: 헤러틱
   개봉일: 2024-10-31
   평점: 7.201
   줄거리: 콜로라도의 작은 마을을 방문한 두 명의 젊은 몰몬교 여성 선교사들이 주민에게 복음을 전파하기 위해 집집마다 방문 중에 매력적인 리드 씨라는 인물을 만나게 되고, 그의 집에서 예상치 못한 위험에 휩싸이게 된다.

3. 제목: 에이리언: 로물루스
   개봉일: 2024-08-13
   평점: 7.228
   줄거리: 2142년, 부모 세대가 맞닥뜨렸던 암울한 미래를 피하려는 청년들이 더 나은 삶을 찾기 위해 식민지를 떠날 계획을 세운다. 하지만 버려진 우주 기지 로물루스에 도착한 이들은 악몽과도 같은 에이리언의 무자비한 공격에 쫓기기 시작한다. 그 누구도 그들의 절규를 들을 수 없는 우주 한가운데, 생존을 위한 치열한 사투를 벌여야 하는데...
